In [12]:
import random

import cv2
import pandas as pd
import numpy as np
import vk_api
import urllib
import requests
import sqlite3

In [13]:
with open('config') as file:
    a = file.readline().split()
    number, password, access_token, *_ = a
vk_session = vk_api.VkApi(number, password, token=access_token)
vk_session.auth()

vk = vk_session.get_api()

In [14]:
group_domain = 'vezdekod'
group_id = -197700721

In [15]:
info = vk.photos.getAlbums(owner_id=group_id)

memesAlb = []
for item in info['items']:
    if 'мем' in item['title']:
        memesAlb.append(item['id'])

photos = []

for alb in memesAlb:
    photosInfo = vk.photos.get(owner_id=group_id, album_id=alb, extended=1, count=100)
    for photo in photosInfo['items']:
        photos.append((photo['user_id'], photo['sizes'][-1]['url'], photo['likes']['count']))

In [16]:
ids = []
paths = []
user_ids = []
likess = []
for n, (user, img, likes) in enumerate(photos):
    ids.append(n)
    paths.append(f'{n}.jpg')
    user_ids.append(user)
    likess.append(likes)
    img_data = requests.get(img).content
    with open(f'photos/{n}.jpg', 'wb') as handler:
        handler.write(img_data)

In [17]:
group_domain = 'sciencemem'
group_id = -159146575

In [18]:
offs = 0
i = 186
while i < 1186:
    info = vk.wall.get(domain=group_domain, extended=1, count=100, offset=offs)
    for post in info['items']:
        if 'attachments' in post.keys() and len(post['attachments']) == 1 and 'photo' in post['attachments'][0].keys():
            img_data = requests.get(post['attachments'][0]['photo']['sizes'][-1]['url']).content
            with open(f'photos/{i}.jpg', 'wb') as handler:
                handler.write(img_data)
            ids.append(i)
            user_ids.append(post['attachments'][0]['photo']['user_id'])
            paths.append(f'{i}.jpg')
            likess.append(post['likes']['count'])
            i+=1
    offs += 100

In [20]:
con = sqlite3.connect('data.db')
cur = con.cursor()
cur.execute("""CREATE TABLE IF NOT EXISTS memes(
   id INT PRIMARY KEY,
   path TEXT,
   user INT,
   likes INT);
   """)
cur.execute("""CREATE TABLE IF NOT EXISTS whitelist(id INT PRIMARY KEY);""")
cur.execute("""CREATE TABLE IF NOT EXISTS stat(is_stat INT);""")

In [21]:
cur.execute("SELECT is_stat FROM stat;")
if not len(cur.fetchall()):
    cur.execute("INSERT INTO stat VALUES(0);")
con.commit()

In [23]:
for i in range(len(ids)):
    cur.execute("INSERT OR IGNORE INTO memes VALUES(?, ?, ?, ?);", (ids[i], paths[i], user_ids[i], likess[i]))

In [34]:
def like(id):
    cur.execute(f"SELECT likes from memes WHERE id = {id};")
    likes = cur.fetchall()[0][0]
    cur.execute(f"UPDATE memes SET likes = {likes + 1} WHERE id = {id};")
    cur.execute("SELECT * FROM stat")
    is_stat = cur.fetchall()[0][0]
    if is_stat:
        cur.execute("SELECT likes FROM memes")
        alllikes = [i[0] for i in cur.fetchall()]
        alllikes = sorted(alllikes)[::-1]
        top_likes = alllikes[:5]
        top_memes = []
        cur.execute(f"SELECT * FROM memes where likes = {top_likes[0]};")
        top_memes.append(cur.fetchall())
        cur.execute(f"SELECT * FROM memes where likes = {top_likes[1]};")
        top_memes.append(cur.fetchall())
        cur.execute(f"SELECT * FROM memes where likes = {top_likes[2]};")
        top_memes.append(cur.fetchall())
        cur.execute(f"SELECT * FROM memes where likes = {top_likes[3]};")
        top_memes.append(cur.fetchall())
        cur.execute(f"SELECT * FROM memes where likes = {top_likes[4]};")
        top_memes.append(cur.fetchall())
        print("TOP 5 MEMES:")
        for i, meme in enumerate(top_memes):
            print(i + 1, ": ", meme)
        cur.execute(f"SELECT * FROM memes WHERE id = {id};")
        print(cur.fetchall())
    con.commit()

In [25]:
def new_mem_pokaz(idx):
    cur.execute("SELECT likes FROM memes;")
    idxs = [i[0] for i in cur.fetchall()]
    _len = len(idxs)
    con.commit()
    while True:
        mx = max(idxs) + 1
        arr = np.array([mx] * _len)
        weights = arr - idxs
        mx = max(weights)
        weights[idx] = int(2 * mx)
        s = np.sum(weights)
        weights = weights / s
        idxx = np.random.choice(range(_len), p=weights)
        cur.execute(f"SELECT path FROM memes WHERE id = {idxx};")
        title = cur.fetchall()[0][0]
        con.commit()
        image = cv2.imread(f'photos/{title}')
        cv2.imshow('', image)
        while True:
            key = cv2.waitKey(1)
            if key == ord('l'):
                like(idxx)
                idxs[idxx] += 1
                break
            elif key == ord('s'):
                break
            elif key == ord('0'):
                return
    cv2.destroyAllWindows()

In [26]:
def add_to_whitelist(id):
    cur.execute(f"INSERT OR IGNORE INTO whitelist VALUES({id});")
    con.commit()

In [27]:
add_to_whitelist(vk.account.getProfileInfo()['id'])

In [28]:
def stat():
    cur.execute("SELECT * FROM whitelist;")
    whitelist = [i[0] for i in cur.fetchall()]
    if vk.account.getProfileInfo()['id'] in whitelist:
        cur.execute("UPDATE stat SET is_stat = 1 WHERE is_stat = 0")
        con.commit()

In [29]:
def stop_stat():
    cur.execute("UPDATE stat SET is_stat = 0 WHERE is_stat = 1")
    con.commit()

In [30]:
stat()

In [35]:
new_mem_pokaz(0)

TOP 5 MEMES:
1 :  [(186, '186.jpg', 100, 33057)]
2 :  [(220, '220.jpg', 100, 30193)]
3 :  [(351, '351.jpg', 100, 28222)]
4 :  [(259, '259.jpg', 100, 27117)]
5 :  [(285, '285.jpg', 100, 22815)]
[(471, '471.jpg', 100, 2565)]
TOP 5 MEMES:
1 :  [(186, '186.jpg', 100, 33057)]
2 :  [(220, '220.jpg', 100, 30193)]
3 :  [(351, '351.jpg', 100, 28222)]
4 :  [(259, '259.jpg', 100, 27117)]
5 :  [(285, '285.jpg', 100, 22815)]
[(458, '458.jpg', 100, 2201)]
TOP 5 MEMES:
1 :  [(186, '186.jpg', 100, 33057)]
2 :  [(220, '220.jpg', 100, 30193)]
3 :  [(351, '351.jpg', 100, 28222)]
4 :  [(259, '259.jpg', 100, 27117)]
5 :  [(285, '285.jpg', 100, 22815)]
[(168, '168.jpg', 163900540, 3)]
TOP 5 MEMES:
1 :  [(186, '186.jpg', 100, 33057)]
2 :  [(220, '220.jpg', 100, 30193)]
3 :  [(351, '351.jpg', 100, 28222)]
4 :  [(259, '259.jpg', 100, 27117)]
5 :  [(285, '285.jpg', 100, 22815)]
[(576, '576.jpg', 100, 3357)]
TOP 5 MEMES:
1 :  [(186, '186.jpg', 100, 33057)]
2 :  [(220, '220.jpg', 100, 30193)]
3 :  [(351, '351.jpg

KeyboardInterrupt: 